In [6]:
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//packages_import.ipynb"    #import all necessary packages - numpy, pandas etc
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//parameters_class.ipynb"                                                                                                  #import the parameters class and the base paramters object
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//simulation_class.ipynb"   
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//outcome_stats_and_plots.ipynb"                                                                                               #import the class that runs the simulation                                                                                            #import the class that runs the simulation                                                                                         #import the class that runs the simulation                                                                                            #import the class that runs the simulation
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations//Stable_Ecological_Competition//base_parameters_2.ipynb" import base_parameters
%run "D://Personal Projects//Species_Discrimination_Evolution_Coexistence//Revised_equations/Stable_Ecological_Competition//base_parameters_2.ipynb" import Parameters_for_Analysis

In [7]:
def plot_2d_sensitivity_analysis_initial_pop_vs_evolvability(N2_N1_ratios, conspecfic_mating_rate_multiplier_ratios, final_outcomes_matrix):
    color_map = {
        "sp 2 (high-genetic-var) wins, sp 1 (low-genetic-var) extinct": 0,
        "sp 1 (low-genetic-var) wins, sp 2 (high-genetic-var) extinct": 1,
        "both extinct": 2,
        "both coexist": 3
    }

    reverse_color_map = {v: k for k, v in color_map.items()}

    # Convert the final outcomes to numerical values
    numerical_matrix = np.vectorize(color_map.get)(final_outcomes_matrix)

    # Create the heatmap
    heatmap = go.Heatmap(
        z=numerical_matrix.T,  # Transpose the matrix to match the correct axis
        x=N2_N1_ratios,
        y=conspecfic_mating_rate_multiplier_ratios,
        colorscale=[[0, 'red'], [1/3, 'blue'], [2/3, 'black'], [1, 'green']],
        showscale = False,
        colorbar=dict(
            tickvals=[0, 1, 2, 3],
            ticktext=list(color_map.keys()),
        )
    )

    fig = go.Figure(data=[heatmap])

    legend_labels = [
        ("Slow evolver extinct", 'red'),
        ("Fast evolver extinct", 'blue'),
        ('Both coexist', 'green'),
        ('Both extinct', 'black')
    ]

    for label, color in legend_labels:
        fig.add_trace(go.Scatter(
            x=[None], y=[None], mode='markers',
            marker=dict(color=color),
            showlegend=True,
            name=label
        ))



    fig.update_layout(
        xaxis=dict(
            title= 'Ratio of conspecific mating rate multiplier [sp.2/sp.1]',
            type='log',
            tickvals= [-2, -1, 0, 1, 2],
            ticktext = ["-2", "-1"," 0", "1","2"],
        ),
        yaxis=dict(
            title='Evolvability Ratio [sp.2/sp.1] (log-scale)',
            type='log',
            tickvals=[1, 10, 100, 1000],
            ticktext=['1', '10', '100', '1000']
        ),
        template='plotly',
        width=1000,
        height=800,

 
    )
    

    pio.show(fig)

def generate_parameters_with_varied_parameters(conspecfic_mating_rate_multiplier_ratios, V_A2_V_A1_ratios):
    parameters_list = []
    for conspecfic_mating_rate_multiplier in conspecfic_mating_rate_multiplier_ratios:
        for V_A2_V_A1 in V_A2_V_A1_ratios:
            parameters_dict = Parameters_for_Analysis(base_parameters=base_parameters, V_A1=0.01, V_A2=0.01).__dict__.copy()
            parameters_dict['conspecfic_mating_rate_multiplier_sp_2'] = parameters_dict['conspecfic_mating_rate_multiplier_sp_1'] * conspecfic_mating_rate_multiplier
            parameters_dict['V_A2'] = parameters_dict['V_A1'] * V_A2_V_A1
            parameters = Parameters_for_Analysis(base_parameters=base_parameters, **parameters_dict)
            parameters_list.append(parameters)
    return parameters_list

def run_sensitivity_analysis(parameters_list):
    final_outcomes_list = []
    for parameters in parameters_list:
        final_outcome_instance = run_simulation_and_get_outcome_instance(parameters)
        final_outcomes_list.append(final_outcome_instance.outcome)
    return final_outcomes_list


In [8]:

def main():

    conspecfic_mating_rate_multiplier_powers = np.arange(-1, 1, 0.25)
    conspecfic_mating_rate_multiplier_ratios = np.power(10.0, conspecfic_mating_rate_multiplier_powers)
    # print(conspecfic_mating_rate_multiplier_ratios)

    V_A2_V_A1_powers = np.arange(0, 2, 0.0625)
    V_A2_V_A1_ratios = np.power(10.0, V_A2_V_A1_powers)
    
    parameters_list = generate_parameters_with_varied_parameters(conspecfic_mating_rate_multiplier_ratios, V_A2_V_A1_ratios)
    final_outcomes_list = run_sensitivity_analysis(parameters_list)
    
    final_outcomes_matrix = np.array(final_outcomes_list).reshape(len(conspecfic_mating_rate_multiplier_ratios), len(V_A2_V_A1_ratios))
    
    plot_2d_sensitivity_analysis_initial_pop_vs_evolvability(conspecfic_mating_rate_multiplier_ratios, V_A2_V_A1_ratios, final_outcomes_matrix)

if __name__ == "__main__":
    main()


Result, 30 times evslowrr olvabilty can reverse outcome

Interpretation: If interspecific competetion > intraspecific competition. The resource competetion accelerates the positive frequency dependance. If interspecific competetion < interpsecific competetin, the resource competetion decelerates the positive frequency depence.

